In [2]:
import requests
import pandas as pd
import csv
url='https://testiws.ximad.com/export/events.csv.gz'
filename = url.split("/")[-1]
with open(filename, "wb") as f:
    r = requests.get(url)
    f.write(r.content)

In [170]:
import gzip
import pandas as pd
with gzip.open('events.csv.gz') as f:
    data = pd.read_csv(f)

data.head()

,event_time,event_name,user_id,event_value
0,2021-09-29 17:58:14,game_start,9586e17375389649,NaN
1,2021-09-23 16:02:12,game_finish,201bf35f4bb535fd,NaN
2,2021-09-25 09:58:37,game_finish,700eeb451fbf5197,NaN
3,2021-09-15 20:46:00,game_start,df651b8bdd042fd6,NaN
4,2021-09-09 22:22:06,game_finish,e84c3f343a5a66df,NaN


In [171]:
data_sorted=data.sort_values(by=['event_time'])
data_sorted['just_date'] = pd.to_datetime(data_sorted['event_time']).dt.date
data_launch=data_sorted[data_sorted.event_name=='launch']
data_launch=data_launch[['just_date','user_id']]

In [172]:
#unique users in day
unique_users=data_launch.groupby(['just_date']).nunique('user_id')
unique_users=unique_users.rename(columns={'user_id': 'users_count'})
unique_users.reset_index(level=0, inplace=True)

In [176]:
#purchases in day
data_purchased=data_sorted[data_sorted.event_name=='purchase']
data_purchased=data_purchased[['just_date','event_value']]
daily_income=data_purchased.groupby(['just_date']).sum()
daily_income=daily_income.rename(columns={'event_value': 'daily_income'})
daily_income.reset_index(level=0, inplace=True)


In [179]:
df_result=unique_users.merge(daily_income, how='inner', on='just_date')#.drop(columns=['index'])

In [180]:
#computing arpdau as shown in https://www.applovin.com/blog/what-is-arpdau/
df_result['ARPDAU']=df_result.apply(lambda row: row.daily_income / row.users_count, axis=1)

In [183]:
#Рассчитанная метрика по среднему доходу с одного уникального пользователя за день
df_result

,just_date,users_count,daily_income,ARPDAU
0,2021-09-07,14686,1010.0,0.068773
1,2021-09-08,14482,1390.0,0.095981
2,2021-09-09,14574,1270.0,0.087141
3,2021-09-10,14537,1630.0,0.112128
4,2021-09-11,13637,1940.0,0.142260
5,2021-09-12,14220,1890.0,0.132911
6,2021-09-13,14607,1190.0,0.081468
7,2021-09-14,14539,790.0,0.054337
8,2021-09-15,14276,1340.0,0.093864
9,2021-09-16,14300,990.0,0.069231


<b>Sql запрос для рассчета ARPDAU по таблице в бд:</b>
<br>
select t1.DateOnly, income/users ARPDAU from
<br>
(
<br>
select DATE(date) DateOnly, count(distinct user_id) users from events
<br>
where event_name='launch'
<br>
group by DATE(date)
<br>
) t1
<br>
join 
<br>
(
<br>
select DATE(date) DateOnly, sum(event_value) income from events
<br>
where event_name='purchase'
<br>
group by DATE(date)
<br>
) t2
<br>
on t1.DateOnly=t2.DateOnly


Далее дополнительно решил создать табличку в постгрес, выгрузить в нее данные и выполнить запрос, стало интересно сравнить с тем, что получилось на pandas

In [59]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [161]:
import psycopg2
conn = psycopg2.connect("host=localhost dbname=DbTest user=postgres  password=123")

In [20]:
#create table in database
database = psycopg2.connect (database = "DbTest", user="postgres", password="123", host="localhost", port="5432")
cursor = database.cursor()
delete = """Drop table if exists events"""

mydata = cursor.execute(delete)

cursor.execute("""CREATE TABLE events(
id SERIAL PRIMARY KEY,
date timestamp,
event_name varchar(30),
user_id varchar(30),
event_value decimal(10,2)
);""")

In [21]:
data.shape

(3966791, 4)

In [22]:
%%time
#inserting data in table
i=0
for index, row in data.iterrows():
    #print(row['event_time'])
    cursor.execute("INSERT INTO events (date, event_name, user_id,event_value)"\
                "VALUES (%s,%s,%s,%s)",(row['event_time'],row['event_name'],row['user_id'],row['event_value'],))
    i+=1
    if i%100000==0:
        print(i)
        


cursor.close()
database.commit()
database.close()

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
Wall time: 34min 18s


In [162]:
#select ARPDAU from database
conn2=psycopg2.connect (database = "DbTest", user="postgres", password="123", host="localhost", port="5432")
selCursor=conn.cursor()
query="""
select t1.DateOnly, income/users ARPDAU from
(
select DATE(date) DateOnly, count(distinct user_id) users from events
where event_name='launch'
group by DATE(date)
) t1
join 
(
select DATE(date) DateOnly, sum(event_value) income from events
where event_name='purchase'
group by DATE(date)
) t2
on t1.DateOnly=t2.DateOnly
"""
selCursor.execute(query)
dbQueryResult=selCursor.fetchall()

In [163]:
#parse data into dataframe
df_from_database = pd.DataFrame(columns = ['just_date','ARPDAU'])
i=0
for row in dbQueryResult:
    df_from_database.loc[i]=[row[0],float(row[1])]
    i+=1

In [164]:
df_from_database.head()

,just_date,ARPDAU
0,2021-09-07,0.068773
1,2021-09-08,0.095981
2,2021-09-09,0.087141
3,2021-09-10,0.112128
4,2021-09-11,0.142260


In [169]:
#compare data from database and data computed by pandas
df_from_database.equals(df_result[['just_date','ARPDAU']])

True